In [ ]:
import time
import numpy as np
import tensorflow as tf

import re
from collections import Counter
import random

In [ ]:
# Some helper functions to prepare the data for processing.
# Referenced from https://github.com/priya-dwivedi/Deep-Learning/tree/master/word2vec_skipgram
def preprocess(text):
    
    #Substituted punctuations with tokens.
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' <COLON> ') 
    words = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words

def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: A list where each item is a tuple of (batch of input, batch of target).
    """
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return list(zip(x_batches, y_batches))

def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

In [ ]:
# Reading the data
dataset = r".\NLP\datasets\text8"
with open(dataset) as f:
    data = f.read()

In [ ]:
# Pre processing words. Replaced punctuations with tokens.
# Also words occuring less than 5 times are discarded.
data = preprocess(data)

In [ ]:
# Defining id for words in a dictionary.
vocab_to_int, int_to_vocab = create_lookup_tables(data)

In [ ]:
# Converting the word corpus into int. The int number here is the ID of the word from the dict vocab_to_int
int_words = [vocab_to_int[word] for word in data]

In [ ]:
# Subsampling needs to be done. For that, we need the frequency of the words
threshold = 1e-5  #Threshold is totally user dependent
words_count = Counter(int_words)
total_count = len(int_words)
freq_words = {word: count/total_count for word, count in words_count.items()}
prob_words = {word: 1 - np.sqrt(threshold/freq) for word, freq in freq_words.items()}
training_words = [word for word in int_words if random.random() < (1 - prob_words[word])]

In [ ]:
def get_target(words, index, window_size = 5):
    n = np.random.randint(1,window_size+1)
    start = index - n if (index - n) > 0  else 0
    end = index + n
    words = set(words[start:index] + words[index:end])
    return list(words)

In [ ]:
# Generator to get a batch of words with N previous and N next words, considering a single word. 
def get_batches(words, batch_size, window_size=5):
    n_batches = len(words) // batch_size
    words = words[:batch_size*n_batches]
    for idx in range(0, len(words),batch_size):
        print(idx)
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for j in range(len(batch)):
            batch_x = batch[j]
            batch_y = get_target(batch, j, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x,y 

In [ ]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

In [ ]:
# No of the hidden nodes = 200
n_vocab = len(int_to_vocab)
n_embedding = 200
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)

In [ ]:
# Negative Sampling with limit = 100
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, 
                                      labels, embed,
                                      n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [ ]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # Cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [ ]:
epochs = 100
batch_size = 1000
window_size = 10

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph,config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(training_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, r"F:\studies\Machine_Learning_study\MACHINE_LEARNING_COMPETITIONS\password_rnn\models\skipgram\text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)